# SA-CCR Euler allocation of an exemplary multi asset class portfolio

We want to show at an example, how the Euler allocation of trades can be summed up to subportfolios to still accurately represent the risk dynamics of the allocated risk measure.

We combine the exemplary equity portfolio with the exemplary rates portfolio set up in the previous sections within a joint portfolio.

In [1]:
import QuantLib as ql
from allocation.Enums import FdApproach2
from allocation.eulerAllocator import EulerAllocator
from collateralAgreement.collateralAgreement import CollateralAgreement, InitialMargining, Margining
from instruments.equity_instruments.equityOption import EquityOption
from instruments.interestRateInstrument.irs import IRS
from instruments.interestRateInstrument.swaption import Swaption
from jupyterUtils import export
from marketdata.interestRateIndices import InterestRateIndex
from sa_ccr.sa_ccr import SA_CCR
from utilities.Enums import SwapDirection, Stock, TradeType, TradeDirection, AssetClass
import pandas as pd

asdf =1

In [2]:
eq_opt_ads_call = EquityOption(underlying=Stock.ADS,
                               maturity=ql.Period(1, ql.Years),
                               notional=2000000,
                               tradeType=TradeType.CALL,
                               tradeDirection=TradeDirection.LONG)

eq_opt_ads_put = EquityOption(underlying=Stock.ADS,
                              maturity=ql.Period(1, ql.Years),
                              notional=3000000,
                              tradeType=TradeType.PUT,
                              tradeDirection=TradeDirection.LONG)

eq_opt_dbk_call = EquityOption(underlying=Stock.DBK,
                               maturity=ql.Period(1, ql.Years),
                               notional = 10000000,
                               tradeType=TradeType.CALL,
                               tradeDirection=TradeDirection.LONG)

payer_usd_6Y = IRS(notional = 1000000000,
                   swapDirection=SwapDirection.PAYER,
                   timeToSwapStart=ql.Period(2, ql.Days),
                   timeToSwapEnd=ql.Period(6, ql.Years),
                   index = InterestRateIndex.USDLIBOR3M)

receiver_eur_6Y = IRS(notional=180000000,
                      swapDirection=SwapDirection.RECEIVER,
                      timeToSwapStart=ql.Period(2,ql.Days),
                      timeToSwapEnd=ql.Period(6,ql.Years),
                      index=InterestRateIndex.EURIBOR6M)

ul_swap = IRS(notional=500000000,
              swapDirection=SwapDirection.RECEIVER,
              timeToSwapStart=ql.Period(1, ql.Years),
              timeToSwapEnd=ql.Period(6, ql.Years),
              index = InterestRateIndex.USDLIBOR3M)
rec_swaption_1_6 = Swaption(underlyingSwap=ul_swap,
                            optionMaturity=ql.Period(1, ql.Years))

In [3]:
trades = [eq_opt_ads_call,
          eq_opt_ads_put,
          eq_opt_dbk_call,
          payer_usd_6Y,
          receiver_eur_6Y,
          rec_swaption_1_6]

joint_ca_vm_only = CollateralAgreement(margining=Margining.MARGINED,
                                       initialMargining=InitialMargining.NO_IM)
joint_ca_vm_only.link_sa_ccr_instance(SA_CCR(joint_ca_vm_only))
joint_ca_vm_only.add_trades(trades)

joint_ca_vm_and_im = CollateralAgreement(margining=Margining.MARGINED,
                                         initialMargining=InitialMargining.SIMM)
joint_ca_vm_and_im.link_sa_ccr_instance(SA_CCR(joint_ca_vm_and_im))
joint_ca_vm_and_im.add_trades(trades)

And perform an Euler allocation using a central difference approach.

In [4]:
eulerAllocator_vm = EulerAllocator(joint_ca_vm_only)
eulerAllocator_vm.fdApproach2 = FdApproach2.Central
eulerAllocator_vm_and_im = EulerAllocator(joint_ca_vm_and_im)
eulerAllocator_vm_and_im.fdApproach2 = FdApproach2.Central

alloc_ead_under_vm = eulerAllocator_vm.allocate_ead()
alloc_ead_under_vm_and_im = eulerAllocator_vm_and_im.allocate_ead()
alloc_im = eulerAllocator_vm_and_im.allocate_im()

We then display the results on a trade level:

In [5]:
def trrep(t):
    import ast
    if t.assetClass == AssetClass.IR:
        dic = ast.literal_eval(str(t))
        if dic['Instrument'] in ['IRS','OIS']:
            return '{:.0f}Mn '.format(dic['Notional']/1000000)+t.index.name+' '+t.swapDirection.name+' '+dic['Instrument']+' '+str(t.ql_timeToSwapEnd)
        else:
            return '{:.0f}Mn '.format(t.underlying_swap.notional/1000000)+t.underlying_swap.index.name+' '+t.underlying_swap.swapDirection.name+' '+dic['Instrument']+' '+str(t.underlying_swap.ql_timeToSwapStart) + ' to ' + str(t.underlying_swap.ql_timeToSwapEnd)
    else:
        dic = ast.literal_eval(str(t))
        return '{:.0f}Mn '.format(dic['Notional']/1000000)+t.underlying.name+' '+dic['TradeType']+' ' +str(t.ql_maturity)

In [6]:
pd.options.display.float_format = '{:,.0f} USD'.format
data = {'Title':[trrep(eq_opt_ads_call),
                 trrep(eq_opt_ads_put),
                 trrep(eq_opt_dbk_call),
                 trrep(receiver_eur_6Y),
                 trrep(payer_usd_6Y),
                 trrep(rec_swaption_1_6),
                 'Overall Portfolio'],
        'Allocated EAD VM only':[alloc_ead_under_vm[eq_opt_ads_call],
                                 alloc_ead_under_vm[eq_opt_ads_put],
                                 alloc_ead_under_vm[eq_opt_dbk_call],
                                 alloc_ead_under_vm[receiver_eur_6Y],
                                 alloc_ead_under_vm[payer_usd_6Y],
                                 alloc_ead_under_vm[rec_swaption_1_6],
                                 sum(alloc_ead_under_vm.values())],
        'Allocated IM':[alloc_im[eq_opt_ads_call],
                        alloc_im[eq_opt_ads_put],
                        alloc_im[eq_opt_dbk_call],
                        alloc_im[receiver_eur_6Y],
                        alloc_im[payer_usd_6Y],
                        alloc_im[rec_swaption_1_6],
                        sum(alloc_im.values())],
        'Allocated EAD VM + IM':[alloc_ead_under_vm_and_im[eq_opt_ads_call],
                                 alloc_ead_under_vm_and_im[eq_opt_ads_put],
                                 alloc_ead_under_vm_and_im[eq_opt_dbk_call],
                                 alloc_ead_under_vm_and_im[receiver_eur_6Y],
                                 alloc_ead_under_vm_and_im[payer_usd_6Y],
                                 alloc_ead_under_vm_and_im[rec_swaption_1_6],
                                 sum(alloc_ead_under_vm_and_im.values())]}
df = pd.DataFrame.from_dict(data)
df = df.set_index('Title')
df.index.name = None
df

,Allocated EAD VM only,Allocated IM,Allocated EAD VM + IM
2Mn ADS Call 1Y,"4,567,360 USD","4,197,013 USD","1,472,268 USD"
3Mn ADS Put 1Y,"-2,602,617 USD","5,969,992 USD","-1,515,288 USD"
10Mn DBK Call 1Y,"8,265,308 USD","17,384,508 USD","1,872,447 USD"
180Mn EURIBOR6M RECEIVER IRS 6Y,"1,957,315 USD","172,265 USD","762,482 USD"
1000Mn USDLIBOR3M PAYER IRS 6Y,"10,873,970 USD","27,859,887 USD","2,059,910 USD"
500Mn USDLIBOR3M RECEIVER Swaption 1Y to 6Y,"-1,773,170 USD","6,763,936 USD","-1,250,488 USD"
Overall Portfolio,"21,288,165 USD","62,347,602 USD","3,401,332 USD"


We can aggregate the results across the asset classes.

In [7]:
sum([alloc_ead_under_vm[t] for t in alloc_ead_under_vm.keys() if t.assetClass == AssetClass.EQ])

10230051.00119859

In [8]:
pd.options.display.float_format = '{:,.0f} USD'.format
data = {'Title':['Equity Subportfolio',
                 'Rates Subportfolio'],
        'Allocated EAD VM only':[sum([alloc_ead_under_vm[t] for t in alloc_ead_under_vm.keys() if t.assetClass == AssetClass.EQ]),
                                 sum([alloc_ead_under_vm[t] for t in alloc_ead_under_vm.keys() if t.assetClass == AssetClass.IR])],
        'Allocated IM':[sum([alloc_im[t] for t in alloc_im.keys() if t.assetClass == AssetClass.EQ]),
                        sum([alloc_im[t] for t in alloc_im.keys() if t.assetClass == AssetClass.IR])],
        'Allocated EAD VM + IM':[sum([alloc_ead_under_vm_and_im[t] for t in alloc_ead_under_vm_and_im.keys() if t.assetClass == AssetClass.EQ]),
                        sum([alloc_ead_under_vm_and_im[t] for t in alloc_ead_under_vm_and_im.keys() if t.assetClass == AssetClass.IR])]
        }
df = pd.DataFrame.from_dict(data)
df = df.set_index('Title')
df.index.name = None
df

,Allocated EAD VM only,Allocated IM,Allocated EAD VM + IM
Equity Subportfolio,"10,230,051 USD","27,551,513 USD","1,829,428 USD"
Rates Subportfolio,"11,058,114 USD","34,796,088 USD","1,571,904 USD"


This result is discussed further in section \todo{Reference required}.

We can aggregate across the asset classes to confirm that no hedges between the asset classes take place as the results are the same as in sections \ref{sa-ccr-euler-allocation-of-exemplary-equity-portfolio} and \ref{euler-allocation-of-an-exemplary-rates-portfolio}.

In [9]:
export("SA-CCR Euler allocation of an examplary multi asset class portfolio.ipynb")